In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import csv

###PART I: Data cleaning

#Main file:
with open('/Users/macf/Downloads/DataAnalysisProjects/transactions_202203281842.csv') as csvDataFile:

    #Read file as csv file and frame it
    df=pd.read_csv(csvDataFile, delimiter=',')
                         
    #Drop rows with negative values
    df=df[df.select_dtypes(include=[np.number]).ge(0).all(1)]
    
    #Drop rows with zeros
    df1=df.loc[~(df==0).all(axis=1)]
    
    #Check data types
    #print(df.dtypes)
    
    #Check the different unique values in column currency
    #df1=df.currency.unique()
    
    #Delet the duplicates in the currency column (info based on Dbeaver visualisation of the db).
    df = df[df.currency.str[-1].eq("\n")]
    
    #df1=df.currency.unique()

    #Conversion of currencies to INR:
    
    #1. Index of column with containing string USD in column currency
    dfUSD=df[df["currency"].str.contains("USD")]
    dfIndex=dfUSD.index
    
    #2. Update prices with conversion rate
    for i in dfIndex:
        df.at[i,'sales_amount']= float(df.at[i,'sales_amount'])*75.78
    
    #check updated values
    #d=df.loc[dfindex]['sales_amount']
    
    
    #Update ticker
    df.replace("USD\n", "INR", inplace=True)
    df.replace("INR\n", "INR", inplace=True)
    
    #df=df.currency.unique()
    print(df)
    
    
    ##Save to csv
    #df.to_csv('/Users/macf/Downloads/DataAnalysisProjects/transactions_clean.csv')

       product_code customer_code market_code  order_date  sales_qty  \
281         Prod001        Cus001     Mark001  2017-10-10        100   
283         Prod002        Cus003     Mark003  2018-04-06          1   
284         Prod002        Cus003     Mark003  2018-04-11          1   
285         Prod002        Cus004     Mark003  2018-06-18          6   
286         Prod003        Cus005     Mark004  2017-11-20         59   
...             ...           ...         ...         ...        ...   
150278      Prod339        Cus005     Mark004  2019-04-18          1   
150279      Prod339        Cus020     Mark004  2019-04-23          1   
150280      Prod339        Cus007     Mark004  2019-04-23          1   
150281      Prod339        Cus006     Mark004  2019-04-24          7   
150282      Prod339        Cus032     Mark009  2019-04-24          3   

        sales_amount currency  
281          41241.0      INR  
283            875.0      INR  
284            583.0      INR  
285    

In [19]:
#We finished to clean the main file. Now we buid the star schema by connecting matching infos from other tabs into the main dataframe (new columns) 

#Join 1
with open('/Users/macf/Downloads/DataAnalysisProjects/cleaned/markets_clean.csv') as csvDataFile:
    dfMarkets=pd.read_csv(csvDataFile, delimiter=',')
    
    #Visulaize the data:
    
    #print(dfMarkets.columns)
    #print(dfMarkets.dtypes)
    #print(dfMarkets.markets_code)
    
    #Harmonize names of the colums
    dfMarkets=dfMarkets.rename(columns={"markets_code":"market_code"})
 
    df3 = df.merge(dfMarkets, how='left', on='market_code')
   
    print(df3)

       product_code customer_code market_code  order_date  sales_qty  \
0           Prod001        Cus001     Mark001  2017-10-10        100   
1           Prod002        Cus003     Mark003  2018-04-06          1   
2           Prod002        Cus003     Mark003  2018-04-11          1   
3           Prod002        Cus004     Mark003  2018-06-18          6   
4           Prod003        Cus005     Mark004  2017-11-20         59   
...             ...           ...         ...         ...        ...   
149996      Prod339        Cus005     Mark004  2019-04-18          1   
149997      Prod339        Cus020     Mark004  2019-04-23          1   
149998      Prod339        Cus007     Mark004  2019-04-23          1   
149999      Prod339        Cus006     Mark004  2019-04-24          7   
150000      Prod339        Cus032     Mark009  2019-04-24          3   

        sales_amount currency  Unnamed: 0 markets_name   zone  
0            41241.0      INR           0      Chennai  South  
1      

In [20]:
#Join 2: create a new column with costumer_names from costumers tab in TXS

#Import the costumer file and connect it to df (df. costumer_code)
with open('/Users/macf/Downloads/DataAnalysisProjects/cleaned/customers_202203290152.csv') as csvDataFile:
    dfCostumers=pd.read_csv(csvDataFile, delimiter=',')   
    #print(dfCostumers)
    
    df4 = df3.merge(dfCostumers, how='left', on='customer_code')
    print(df4)

       product_code customer_code market_code  order_date  sales_qty  \
0           Prod001        Cus001     Mark001  2017-10-10        100   
1           Prod002        Cus003     Mark003  2018-04-06          1   
2           Prod002        Cus003     Mark003  2018-04-11          1   
3           Prod002        Cus004     Mark003  2018-06-18          6   
4           Prod003        Cus005     Mark004  2017-11-20         59   
...             ...           ...         ...         ...        ...   
149996      Prod339        Cus005     Mark004  2019-04-18          1   
149997      Prod339        Cus020     Mark004  2019-04-23          1   
149998      Prod339        Cus007     Mark004  2019-04-23          1   
149999      Prod339        Cus006     Mark004  2019-04-24          7   
150000      Prod339        Cus032     Mark009  2019-04-24          3   

        sales_amount currency  Unnamed: 0 markets_name   zone  \
0            41241.0      INR           0      Chennai  South   
1    

In [21]:
#Join 3: create a new column with product type from costumers tab in TXS

with open('/Users/macf/Downloads/DataAnalysisProjects/cleaned/products_202203290052.csv') as csvDataFile:
    dfProducts=pd.read_csv(csvDataFile, delimiter=',')   
    
    #Check the data:
    #print(dfProducts)
    #dfP=dfProducts.product_type.unique()
    
    df5 = df4.merge(dfProducts, how='left', on='product_code')
    print(df5)

       product_code customer_code market_code  order_date  sales_qty  \
0           Prod001        Cus001     Mark001  2017-10-10        100   
1           Prod002        Cus003     Mark003  2018-04-06          1   
2           Prod002        Cus003     Mark003  2018-04-11          1   
3           Prod002        Cus004     Mark003  2018-06-18          6   
4           Prod003        Cus005     Mark004  2017-11-20         59   
...             ...           ...         ...         ...        ...   
149996      Prod339        Cus005     Mark004  2019-04-18          1   
149997      Prod339        Cus020     Mark004  2019-04-23          1   
149998      Prod339        Cus007     Mark004  2019-04-23          1   
149999      Prod339        Cus006     Mark004  2019-04-24          7   
150000      Prod339        Cus032     Mark009  2019-04-24          3   

        sales_amount currency  Unnamed: 0 markets_name   zone  \
0            41241.0      INR           0      Chennai  South   
1    

In [34]:
#Join 4: create a new column with year from sales_dates tab in TXS

with open('/Users/macf/Downloads/DataAnalysisProjects/cleaned/_date__202203290151.csv') as csvDataFile:
    dfDates=pd.read_csv(csvDataFile, delimiter=',')   
    
    #Check the data:
    #print(dfDates)
    #dfP=dfProducts.product_type.unique()
    
   
    #Harmonize names of the colums
    dfDates=dfDates.rename(columns={"date":"order_date"})
    
    #Join
    df = df5.merge(dfDates, how='left', on='order_date')
    print(df)
    
    #df is cleaned and totally mapped we can save our final version#
    #df.to_csv('/Users/macf/Downloads/DataAnalysisProjects/cleaned/Final_Cleaned_df.csv')

       product_code customer_code market_code  order_date  sales_qty  \
0           Prod001        Cus001     Mark001  2017-10-10        100   
1           Prod002        Cus003     Mark003  2018-04-06          1   
2           Prod002        Cus003     Mark003  2018-04-11          1   
3           Prod002        Cus004     Mark003  2018-06-18          6   
4           Prod003        Cus005     Mark004  2017-11-20         59   
...             ...           ...         ...         ...        ...   
149996      Prod339        Cus005     Mark004  2019-04-18          1   
149997      Prod339        Cus020     Mark004  2019-04-23          1   
149998      Prod339        Cus007     Mark004  2019-04-23          1   
149999      Prod339        Cus006     Mark004  2019-04-24          7   
150000      Prod339        Cus032     Mark009  2019-04-24          3   

        sales_amount currency  Unnamed: 0 markets_name   zone  \
0            41241.0      INR           0      Chennai  South   
1    

In [23]:
###PART II: Data visualisation

#Sales by costumers name

#New dataframe sales sum grouped by customers
dff0 = df.groupby(["custmer_name"]).sales_amount.sum().reset_index()

#sort 
dff0 = dff0.sort_values('sales_amount', ascending=False)
#fix typo
dff0=dff0.rename(columns={'custmer_name':'customer_name'})

#plot
fig = px.bar(dff0, x="customer_name", y=["sales_amount"], title="Sales Amount by Customers")

fig.show()


In [24]:
#New dataframe sales grouped by markets
dff1 = df.groupby(["markets_name"]).sales_amount.sum().reset_index()
#print(dff1)

#sort 
dff1=dff1.sort_values(by=['sales_amount'])
#dff = dff.sort_values('sales_amount')

#plot yhe bars
fig = px.bar(dff1, x="markets_name", y=["sales_amount"], title="Sales amount by Markets")
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [25]:
dff1 = df.groupby(["custmer_name"]).sales_qty.sum().reset_index()

#sort 
dff1 = dff1.sort_values('sales_qty', ascending=False)
#fix typo
dff1=dff1.rename(columns={'custmer_name':'customer_name'})


import plotly.express as px
#plot
fig = px.bar(dff1, x="customer_name", y=["sales_qty"], title="Sales Qty by Customers")

fig.show()

In [26]:
#New dataframe sales qty grouped by markets
dff2 = df.groupby(["markets_name"]).sales_qty.sum().reset_index()
#print(dff1)

#sort 
dff2=dff2.sort_values(by=['sales_qty'])

#plot the bars
fig = px.bar(dff2, x="markets_name", y=["sales_qty"], title="Sales Qty by Markets")
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [27]:
#New dataframe sales grouped by year
dff3 = df.groupby(["year"]).sales_amount.sum().reset_index()

#sort 
dff3=dff3.sort_values(by=['year'],ascending=True)

#plot the bars
fig = px.line(dff3, x="year", y=["sales_amount"], title="Sales amount per year")

fig.show()

In [29]:
#Top 5 product sold
dff4 = df.groupby(["product_code"]).sales_amount.sum().reset_index()


#sort 
dff4=dff4.sort_values(by=['sales_amount'], ascending= False)
#filter top 5
dff4 = dff4.head(5)
#plot the bars
fig = px.bar(dff4, x="product_code", y=["sales_amount"], title="Top 5 products sold")

fig.show()

In [30]:
#Top 5 customers
dff5 = df.groupby(["custmer_name"]).sales_amount.sum().reset_index()


#sort 
dff5 = dff5.sort_values(by=['sales_amount'], ascending= False)
#filter top 5
dff5 = dff5.head(5)
#fix typo
dff5=dff5.rename(columns={'custmer_name':'customer_name'})

#plot the bars
fig = px.bar(dff5, x="customer_name", y=["sales_amount"], title="Top 5 customers")

fig.show()

In [ ]:
#Project finished